# Ivestment Porfolio Management

## Load investments and print short report

In [ ]:
import sys
import os
import logging
import pandas
import numpy

logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

# Give priority to local packages (not needed in case Robson was installed by pip)
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(''), '..')))

import investorzilla

In [ ]:
# Use data from cache
me=investorzilla.Investor("path/to/investorzilla.yaml")

# # Pass a refreshMap to force refresh data from the Internet or local data sources
# refresh=dict(
#     portfolio=True,
#     benchmarks=False,
#     currency_converters=False
# )

# me=investorzilla.Investor(
#     "../investor_ui_config.yaml",
#     refreshMap=refresh #dict(zip(investorzilla.investor.domains,len(investorzilla.investor.domains)*[True]))
# )

In [ ]:
me.config

In [ ]:
for b in me.benchmarks:
    display(b['obj'])
    # display(b['obj'].data.head(4))

In [ ]:
me.benchmarks[5]['obj']

## Get list of investment instruments and currencies

In [ ]:
me.portfolio.funds()

## Get compound fund from a few instruments

In [ ]:
me.exchange.currency='BRL'

In [ ]:
# funds=['TraderBot KuCoin']
funds=[]

In [ ]:
# fund=me.portfolio.getFund(currencyExchange=me.exchange)
fund=me.portfolio.getFund(funds,currencyExchange=me.exchange)
fund

In [ ]:
fund.setName(top=3)

## Get current Balance of all instruments

In [ ]:
contrib=0.8

currentBalance=fund.balance.groupby(level=0).last()
currentBalance

currentBalance.columns=currentBalance.columns.droplevel()

(
    currentBalance
    .sort_values(
        fund.exchange.target,
        ascending=False
    )
    .assign(
        cum=lambda table: table[fund.exchange.target].cumsum()
    )
    .style.format(
        {
            me.exchange.currency: "${:,.2f}",
            "cum": "${:,.2f}",
        }
    )

    # .query("cum<{}".format(currentBalance[me.exchange.target].sum()*contrib))
    # .index
)

### Current Balance of only the top 70% of Portfolio

In [ ]:
qcut=0.7

(
    currentBalance
    .sort_values(me.exchange.target,ascending=False)
    .assign(
        cum=currentBalance[me.exchange.target].cumsum()
    )
    .query("cum<{}".format(currentBalance[me.exchange.target].sum()*qcut))
    .style.format(
        {
            me.exchange.currency: "${:,.2f}",
            "cum": "${:,.2f}",
        }
    )
)

### Current Balance of top 3 instruments

In [ ]:
top=3

(
    currentBalance
    .sort_values(me.exchange.target,ascending=False)
    .head(top)
    [[me.exchange.target]]
    .style.format(
        {
            me.exchange.currency: "${:,.2f}",
            "cum": "${:,.2f}",
        }
    )
)

## Compute periodic report

`Fund` class methods `report()` and `periodicReport()` compute all KPIs based on only 3 sources of data:

- Balance
- Ledger
- Benchmark (optional)

`periodicReport()` might get a [Pandas' period alias](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-period-aliases) to return a regular monotonic time series. If not passed, the time series returned is ragged and has as many data points as the original data.

`report()` must get a period pair (defaults to "month & year") so it can acoplate the `periodicReport`s of the short term (month) and long term (year) periods. [Tested and supported period pairs are defined in `Fund.periodPairs`](https://github.com/avibrazil/investorzilla/blob/b9cd73dcefc8ecd79cb7d8c402f0fd1f3b5f88c1/investorzilla/fund.py#L54) and go from very high frequency as "hour of the day" to very long term views as "year & decade".

Pass a list to `kpi` parameter to show only specific KPIs. Avaialble are:

**Source of all information**
* `investorzilla.KPI.BALANCE`\
plain balance
* `investorzilla.KPI.MOVEMENTS`\
money added and removed from fund on period

**Cumulative movements**
* `investorzilla.KPI.SAVINGS`\
cumulative money added and removed from fund

**Rate of accumulated gains**
* `investorzilla.KPI.BALANCE_OVER_SAVINGS`\
balance ➗ savings

**Pure gain, on the period or accumulated**
* `investorzilla.KPI.GAINS`\
cumulative gains
* `investorzilla.KPI.PERIOD_GAIN`\
gain on each period

**Normalization**
* `investorzilla.KPI.SHARES`\
amount of shares
* `investorzilla.KPI.SHARE_VALUE`\
value of a share

**Performance**
* `investorzilla.KPI.RATE_RETURN`\
percentage change of share value

**KPIs related to external sources**
* `investorzilla.KPI.BENCHMARK`\
raw value of the benchmark
* `investorzilla.KPI.BENCHMARK_RATE_RETURN`\
variation of benchmark in relation to the last period
* `investorzilla.KPI.BENCHMARK_EXCESS_RETURN`\
relation between RATE_RETURN and BENCHMARK_RATE_RETURN, on each period

### Month and Year

In [ ]:
fund.periodicReport('M')

In [ ]:
fund.periodicReport('Y')

In [ ]:
import IPython
import numpy

In [ ]:
r=fund.report(benchmark=me.benchmarks[8]['obj'])

IPython.display.HTML(
    r
    .apply(lambda cell: numpy.where(cell<0,"color: red",None), axis=1)
    .set_table_styles([
        dict(selector="td", props="font-size: 0.8em; text-align: right"),
        dict(selector="tr", props="line-height: inherit; padding: 0;")
    ])
    # .applymap(lambda value,props: value<0, props='color:red;')
    .to_html()
)

### Daily

In [ ]:
fund.report('day & week',benchmark=me.benchmarks[2]['obj'], output='flat')

#### Plain daily gains

In [ ]:
me.benchmarks[8]

In [ ]:
fund.report('day & week',benchmark=me.benchmarks[8]['obj'],kpi=[investorzilla.KPI.RATE_RETURN,investorzilla.KPI.BALANCE_OVER_SAVINGS,investorzilla.KPI.BENCHMARK,investorzilla.KPI.BENCHMARK_RATE_RETURN])

In [ ]:
fund.report('day & week',benchmark=me.benchmarks[2]['obj'],kpi=[investorzilla.KPI.PERIOD_GAIN,investorzilla.KPI.GAINS,investorzilla.KPI.RATE_RETURN,investorzilla.KPI.BALANCE_OVER_SAVINGS])

In [ ]:
fund.periodicReport('M',benchmark=me.benchmarks[8]['obj'])

### Week & 4 Weeks
#### Performance report

In [ ]:
# fund=me.portfolio[0]['obj'].getFund(['ShiguBot Binance','ShiguBot MB'],currencyExchange=me.exchange)
fund=me.portfolio.getFund(['TraderBot KuCoin'],currencyExchange=me.exchange)

fund.report(
    period='week & 4 weeks',
    benchmark=me.benchmarks[8]['obj'],
    kpi=[
        investorzilla.KPI.RATE_RETURN,
        investorzilla.KPI.BENCHMARK_RATE_RETURN,
        investorzilla.KPI.BENCHMARK_EXCESS_RETURN,
        investorzilla.KPI.PERIOD_GAIN
    ],
)

#### Wealth Evolution

In [ ]:
fund.report(
    period='week & 4 weeks',
    benchmark=me.benchmarks[9]['obj'],
    kpi=[
        investorzilla.KPI.BALANCE,
        investorzilla.KPI.BALANCE_OVER_SAVINGS,
        investorzilla.KPI.GAINS,
        investorzilla.KPI.SAVINGS,
        investorzilla.KPI.MOVEMENTS
    ],
)

#### Plain weekly gains

In [ ]:
fund.report('week & 4 weeks',benchmark=me.benchmarks[9]['obj'],kpi=[investorzilla.KPI.PERIOD_GAIN])

#### Plain Periodic Report with exquisite periods

In [ ]:
pandas.concat([l], axis=1, keys=[fund.periodicReport('3M').index[0]])

----